In [12]:
from keras.models import Sequential
from keras.layers import Input
from keras.models import Model
from keras.optimizers import Adam
from keras.layers.core import Lambda, Dense
from keras import backend as K
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import load_model
from keras.regularizers import l2
K.set_image_data_format('channels_last')
import csv
import os
import numpy as np
import pandas as pd

In [13]:
def initialize_weights(shape, dtype=np.float32):
    return np.random.normal(loc = 0.0, scale = 1e-2, size = shape)
def initialize_bias(shape, dtype=np.float32):
    return np.random.normal(loc = 0.5, scale = 1e-2, size = shape)
def get_siamese_model():
    left_input = Input(shape=(38,))
    right_input = Input(shape=(38,))
    model = Sequential()
    model.add(Dense(64, activation='relu',
                    input_shape=(38,),
                    kernel_regularizer=l2(1e-3),
                    kernel_initializer=initialize_weights,bias_initializer=initialize_bias))
    
    model.add(Dense(512, activation='relu',
                    kernel_regularizer=l2(1e-3),
                    kernel_initializer=initialize_weights,bias_initializer=initialize_bias))
    
    model.add(Dense(512, activation='relu',
                    kernel_regularizer=l2(1e-3),
                    kernel_initializer=initialize_weights,bias_initializer=initialize_bias))
    
    encoded_l = model(left_input)
    encoded_r = model(right_input)
    L1_layer = Lambda(lambda tensors:K.abs(tensors[0] - tensors[1]))
    L1_distance = L1_layer([encoded_l, encoded_r])
    prediction = Dense(1,activation='sigmoid', bias_initializer=initialize_bias)(L1_distance)
    siamese_net = Model(inputs=[left_input,right_input],outputs=prediction)
    return siamese_net

In [14]:
def make_input(data, support_data):
    input1=[]
    input2=[]
    for i in range(0, len(data)):
        for j in range(0, len(support_data)):
            input1.append(data[i])
        for j in support_data:
            input2.append(j[0:38])

    input1=np.array(input1).astype(np.float32)
    input2=np.array(input2).astype(np.float32)

    data_size=len(data)
    support_data_size=len(support_data)
    return input1, input2, data_size, support_data_size

# 2, 3, 4 WAY LEARNING 

In [30]:
m=pd.read_csv('data/data_by_label_test_unseen/mscan.csv', 'r', delimiter=',').values
a=pd.read_csv('data/data_by_label_test_unseen/apache2.csv', 'r', delimiter=',').values
p=pd.read_csv('data/data_by_label_test_unseen/processtable.csv', 'r', delimiter=',').values
s=pd.read_csv('data/data_by_label_test_unseen/snmpguess.csv', 'r', delimiter=',').values
def predict_acc(*args):
    model=get_siamese_model()
    model=load_model('model/test1.h5',custom_objects={'initialize_weights':initialize_weights, 'initialize_bias':initialize_bias})
    data=args[0]
    for i in range(1, len(args)):
        data=np.concatenate((data, args[i]), axis=0)
    feature=[]
    label=[]
    for i in data:
        feature.append(i[0:38])
        label.append(i[38])
    
    support_datas=pd.read_csv('data/support_data/support_data_one_shot.csv', 'r', delimiter=',').values
    support_data=[]
    label_array=[]
    for i in support_datas:
        if i[38] in label:
            support_data.append(i)
            label_array.append(i[38])
    
    feature=np.reshape(feature, (len(feature), 38)).astype(np.float32)
    input1, input2, data_size, support_data_size=make_input(feature, support_data)
    
    prediction=model.predict([input1, input2])
    prediction=prediction.reshape(data_size, support_data_size)
    
    prediction_best=[]
    for i in prediction:
        prediction_best.append(np.where(min(i)==i)[0][0])
    
    label_num=[]
    for i in label:
        for j in label_array:
            if i==j:
                label_num.append(label_array.index(i))
    
    true=0
    false=0
    for i in range(0, len(prediction_best)):
        if prediction_best[i]==label_num[i]:
            true+=1
        else:
            false+=1
            
    print(true/(true+false))

In [31]:
predict_acc(m, a)
predict_acc(m, p)
predict_acc(m, s)
predict_acc(a, p)
predict_acc(a, s)
predict_acc(p, s)
print()
predict_acc(m, a, p)
predict_acc(m, a, s)
predict_acc(m, p, s)
predict_acc(a, p, s)
print()
predict_acc(m, a, p, s)

0.8742065781881131
0.8251041046995836
0.9811605124340618
0.9036568213783404
0.8885767790262172
0.890748031496063

0.815136476426799
0.8570736434108527
0.8449304174950298
0.8185966913861951

0.8101127682793743


# 1 WAY LEARNING

In [62]:
m=pd.read_csv('data/data_by_label_test_unseen/mscan.csv', 'r', delimiter=',')
a=pd.read_csv('data/data_by_label_test_unseen/apache2.csv', 'r', delimiter=',')
p=pd.read_csv('data/data_by_label_test_unseen/processtable.csv', 'r', delimiter=',')
s=pd.read_csv('data/data_by_label_test_unseen/snmpguess.csv', 'r', delimiter=',')

merge=pd.DataFrame(m, columns=m.columns)
merge=pd.concat([merge, a], axis=0)
merge=pd.concat([merge, p], axis=0)
merge=pd.concat([merge, s], axis=0)
merge=merge.values

feature=[]
label=[]
for i in merge:
    feature.append(i[0:38])
    label.append(i[38])
feature=np.reshape(feature, (len(feature), 38)).astype(np.float32)

support_datas=pd.read_csv('data/support_data/support_data_one_shot.csv', 'r', delimiter=',').values
support_data=[]
label_array=[]
for i in support_datas:
    if i[38] in label:
        support_data.append(i)
        label_array.append(i[38])

input1, input2, data_size, support_data_size=make_input(feature, support_data)

model=get_siamese_model()
model=load_model('model/test1.h5',custom_objects={'initialize_weights':initialize_weights, 'initialize_bias':initialize_bias})
prediction=model.predict([input1, input2])
prediction=prediction.reshape(data_size, support_data_size)

label_num=[]
for i in label:
    for j in label_array:
        if i==j:
            label_num.append(label_array.index(i))
            

for i in range(0, len(prediction[0])):
    true=0
    false=0
    for j in range(0, len(prediction)):
        if label_num[j]==i:
            if prediction[j][i]<=0.5:
                true+=1
            else:
                false+=1
        else:
            if prediction[j][i]>0.5:
                true+=1
            else:
                false+=1
    print(label_array[i], end=' : ')
    print(true/(true+false))

mscan : 0.5405602037104401
apache2 : 0.8177519097853765
processtable : 0.7911967988359403
snmpguess : 0.5707530010913059
